In [58]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from pathlib import Path

In [59]:
file ='AI_Powered_Eduction_system'
def root(root_name:str=file):
    for parent in Path().resolve().parents:
        if parent.name == root_name:
            return(parent) 
ROOT = root()

In [60]:
student_performance = pd.read_csv(ROOT/'data\Train_datasets\Score_Predictor\student_performance_log.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Udith\AppData\Local\Temp\ipykernel_13164\1177100319.py:1: SyntaxWarning: invalid escape sequence '\T'
  student_performance = pd.read_csv(ROOT/'data\Train_datasets\Score_Predictor\student_performance_log.csv')


In [61]:
X_cols=(list(student_performance.columns))
X_cols.remove('score')
x_format = student_performance[X_cols]
x_format = x_format.iloc[:2]
x_format.to_csv(ROOT/'data\Working_Data\score_data_table.csv',index=False)

<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Udith\AppData\Local\Temp\ipykernel_13164\160676375.py:5: SyntaxWarning: invalid escape sequence '\W'
  x_format.to_csv(ROOT/'data\Working_Data\score_data_table.csv',index=False)


In [62]:
student_performance.head()

,Student_ID,skill,Test_ID,Type,Days_prepared,TimeTaken,score
0,1,m-measurement,81,checkboxquestion,63,73,69.0
1,1,m-measurement,80,algebra,55,116,14.0
2,1,m-measurement,26,interfaceradioquestion,31,32,31.0
3,1,m-measurement,16,interfacepopupmenuquestion1,48,36,33.0
4,1,m-measurement,40,algebrafieldquestion1,33,41,41.0


In [63]:
con = sql.connect(ROOT/r'data\Train_datasets\Pass_Fail\AI_Edu_system.db')
cur = con.cursor()

In [64]:
skills = pd.read_sql('SELECT * FROM skills',con)

In [65]:
print(skills.head())

   skill_ID                             skill
0         0                   adding-decimals
1         1                          addition
2         2             algebra symbolization
3         3            algebraic-manipulation
4         4  application: compare expressions


In [66]:
student_performance = student_performance.merge(skills, on='skill')
student_performance = student_performance.drop(columns = ['Student_ID','Test_ID','skill'],axis = 1)
# stud_perf_enc = pd.get_dummies(student_performance,columns=['cluster','Type'],dtype=int)


In [67]:
X_cols=(list(student_performance.columns))
X_cols.remove('score')
X = student_performance[X_cols]
y = student_performance['score']

In [68]:
X

,Type,Days_prepared,TimeTaken,skill_ID
0,checkboxquestion,63,73,48
1,algebra,55,116,48
2,interfaceradioquestion,31,32,48
3,interfacepopupmenuquestion1,48,36,48
4,algebrafieldquestion1,33,41,48
...,...,...,...,...
13995,textfieldquestion,34,76,54
13996,popupmenuquestion,79,109,54
13997,interfaceradioquestion,50,86,54
13998,noprobtype,12,44,54


In [69]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=50)

In [70]:
preprocessor = ColumnTransformer(transformers=[('cat',OneHotEncoder(handle_unknown='ignore'),['skill_ID','Type'])],remainder='passthrough')
preprocessor.fit(x_train)
x_train = preprocessor.transform(x_train)
x_test = preprocessor.transform(x_test)

In [71]:
from sklearn.tree import DecisionTreeRegressor

In [72]:
m = DecisionTreeRegressor()
m.fit(x_train,y_train)

DecisionTreeRegressor()

In [73]:
m.score(x_train,y_train)

0.9997740804896553

In [74]:
y_pred = m.predict(x_test)

In [75]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,root_mean_squared_error
def regression_report(yt,yp):
    report = {'mae':mean_absolute_error(yt,yp).round(2),
              'rmse':root_mean_squared_error(yt,yp).round(2),
              'r2':(r2_score(yt,yp)).__round__(4)}
    return report



In [76]:
print(regression_report(y_test,y_pred))

{'mae': 15.24, 'rmse': 25.08, 'r2': -0.1402}


In [77]:
m=DecisionTreeRegressor(max_depth=3)
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 13.42, 'rmse': 19.4, 'r2': 0.3177}


In [78]:
from sklearn.ensemble import RandomForestRegressor

In [79]:
m=RandomForestRegressor()
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.25, 'rmse': 19.41, 'r2': 0.3173}


In [80]:
m=RandomForestRegressor(n_estimators=500)
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.21, 'rmse': 19.35, 'r2': 0.3216}


In [81]:
m=RandomForestRegressor(n_estimators=100,max_depth=4)
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.47, 'rmse': 18.59, 'r2': 0.3739}


In [82]:
from sklearn.ensemble import GradientBoostingRegressor

In [83]:
gb=GradientBoostingRegressor()
gb.fit(x_train,y_train)
y_pred = gb.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.11, 'rmse': 18.3, 'r2': 0.3932}


In [84]:
m=GradientBoostingRegressor(loss='absolute_error')
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 11.4, 'rmse': 19.4, 'r2': 0.3179}


In [85]:
from xgboost import XGBRegressor

In [86]:
m = XGBRegressor()
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.43, 'rmse': 19.57, 'r2': 0.306}


In [87]:
m = XGBRegressor()
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.43, 'rmse': 19.57, 'r2': 0.306}


In [88]:
from sklearn.ensemble import AdaBoostRegressor

In [89]:
m = AdaBoostRegressor()
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 13.88, 'rmse': 19.16, 'r2': 0.3349}


In [90]:
m = AdaBoostRegressor(n_estimators=100,learning_rate = 0.6)
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 13.87, 'rmse': 19.1, 'r2': 0.3385}


In [91]:
from sklearn.neighbors import KNeighborsRegressor

In [92]:
m = KNeighborsRegressor()
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 13.05, 'rmse': 20.19, 'r2': 0.2614}


In [93]:
knn = KNeighborsRegressor(n_neighbors=20,weights='distance')
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 12.17, 'rmse': 18.58, 'r2': 0.3746}


In [94]:
from sklearn.svm import SVR

In [95]:
m = SVR()
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 11.59, 'rmse': 20.75, 'r2': 0.2194}


In [96]:
m = SVR(kernel='rbf',C=1.8,tol=1)
m.fit(x_train,y_train)
y_pred = m.predict(x_test)
print(regression_report(y_test,y_pred))

{'mae': 11.58, 'rmse': 20.71, 'r2': 0.2228}


In [97]:
pipe = Pipeline(steps=[('preprocessor',preprocessor),('model',gb)])
filename = ROOT/'Models\Score_predictor.sav'
pickle.dump(pipe,open(filename,'wb'))

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Udith\AppData\Local\Temp\ipykernel_13164\2362591657.py:2: SyntaxWarning: invalid escape sequence '\S'
  filename = ROOT/'Models\Score_predictor.sav'
